### pyopencl Python Classical XY-Model

The equations:


$F[\vec \phi]=\int d^2x[(\nabla \vec \phi)^2+V_0(\vec \phi ^2 - 1)^2]$

$\partial _ t \vec \phi = -\Gamma \delta F / \delta \vec \phi$

$ \langle \vec \phi (x,0)\cdot \vec \phi (x',0) \rangle = \Delta \delta (x-x')$


The actual equation that I end up solving:

$\frac {d \vec \phi}{dt} = - \Gamma ( V_0 2 \vec \phi (\vec \phi ^2 - 1) - 2 \nabla ^2 \vec \phi)$

I do this for x and y (two seperate equations) with backwards euler 5-point gradiant stencil as numerical scheme.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from numba import njit, prange
import os
import imageio.v2 as imageio
from IPython.display import Image, display, HTML

In [3]:
import pyopencl as cl

def list_opencl_devices():
    platforms = cl.get_platforms()
    if not platforms:
        print("No OpenCL platforms found.")
        return

    for platform in platforms:
        print(f"Platform: {platform.name} ({platform.vendor})")
        devices = platform.get_devices()
        for device in devices:
            print(f"  Device: {device.name}")
            print(f"    Type: {cl.device_type.to_string(device.type)}")
            print(f"    Max Compute Units: {device.max_compute_units}")
            print(f"    Max Work Group Size: {device.max_work_group_size}")
            print(f"    Global Memory Size: {device.global_mem_size / 1024 / 1024} MB")
            print(f"    Local Memory Size: {device.local_mem_size / 1024} KB")
            print(f"    OpenCL Version: {device.opencl_c_version}")

list_opencl_devices()

Platform: Portable Computing Language (The pocl project)
  Device: cpu-haswell-Intel(R) Core(TM) i5-7400 CPU @ 3.00GHz
    Type: ALL | CPU
    Max Compute Units: 4
    Max Work Group Size: 4096
    Global Memory Size: 5857.82421875 MB
    Local Memory Size: 256.0 KB
    OpenCL Version: OpenCL C 1.2 PoCL
  Device: NVIDIA GeForce GTX 1060 3GB
    Type: ALL | GPU
    Max Compute Units: 9
    Max Work Group Size: 1024
    Global Memory Size: 2998.875 MB
    Local Memory Size: 48.0 KB
    OpenCL Version: OpenCL C 1.2 PoCL


In [4]:
import pyopencl as cl

platforms = cl.get_platforms()
for platform in platforms:
    print("Platform:", platform.name)
    devices = platform.get_devices()
    for device in devices:
        print("Device:", device.name)

Platform: Portable Computing Language
Device: cpu-haswell-Intel(R) Core(TM) i5-7400 CPU @ 3.00GHz
Device: NVIDIA GeForce GTX 1060 3GB


In [5]:
# Constants

V_0 = 1
# Kinetic coefficient (timescale)
Gamma = 0.5
# Determines the order of the initial state
Delta = 1
# Scalar
s = 1
# Size of lattice
N = 20
M = 20
# Timestep
dt = 0.1
# Amount of timesteps I want the simulation to run in units: number divided by size of dt
T = 10

In [6]:
current_dir = os.getcwd()
output_dirs = ['vector_field_images', 'angle_field_images', 'scalar_field_images']
for dir in output_dirs:
    os.makedirs(os.path.join(current_dir, dir), exist_ok=True)

In [7]:
# OpenCL setup
platform = cl.get_platforms()[0]
device = platform.get_devices()[1]
context = cl.Context([device])
queue = cl.CommandQueue(context)

In [8]:
# Kernels
program = cl.Program(context, """
__kernel void create_vector_field(__global float2 *vector_field, int N, int M, float s, int seed) {
    int i = get_global_id(0);
    int j = get_global_id(1);
    int index = i * M + j;

    // Simple LCG for random numbers
    int lcg_a = 1664525;
    int lcg_c = 1013904223;
    seed = seed * (index + 1);
    seed = (lcg_a * seed + lcg_c) % 0x10000;
    float rand_val = (float)seed / 0x10000;

    float angle = rand_val * 2.0f * M_PI;
    vector_field[index].x = s * cos(angle);
    vector_field[index].y = s * sin(angle);
}

__kernel void angle_from_field(__global float2 *vector_field, __global float *angle_field, int N, int M, float s) {
    int i = get_global_id(0);
    int j = get_global_id(1);
    int index = i * M + j;

    angle_field[index] = atan2(vector_field[index].y / s, vector_field[index].x / s);
}

__kernel void scalar_from_field(__global float2 *vector_field, __global float *scalar_field, int N, int M) {
    int i = get_global_id(0);
    int j = get_global_id(1);
    int index = i * M + j;

    scalar_field[index] = sqrt(vector_field[index].x * vector_field[index].x + vector_field[index].y * vector_field[index].y);
}

__kernel void vector_field_derivative(__global float2 *vector_field, __global float2 *vector_field_derived, int N, int M, float V_0, float Gamma) {
    int i = get_global_id(0);
    int j = get_global_id(1);
    int index = i * M + j;

    int up = (i - 1 + N) % N;
    int down = (i + 1) % N;
    int left = (j - 1 + M) % M;
    int right = (j + 1) % M;

    float2 non_linear_term = V_0 * 2.0f * vector_field[index] * (vector_field[index].x * vector_field[index].x + vector_field[index].y * vector_field[index].y - 1.0f);
    float2 laplacian_term = (vector_field[up * M + j] + vector_field[down * M + j] + vector_field[i * M + left] + vector_field[i * M + right] - 4.0f * vector_field[index]);

    vector_field_derived[index] = -Gamma * (non_linear_term - 2.0f * laplacian_term);
}

__kernel void update_vector_field(__global float2 *vector_field, __global float2 *vector_field_derived, float dt, int N, int M) {
    int i = get_global_id(0);
    int j = get_global_id(1);
    int index = i * M + j;

    vector_field[index] += dt * vector_field_derived[index];
}
""").build()

RuntimeError: clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE

Build on <pyopencl.Device 'NVIDIA GeForce GTX 1060 3GB' on 'Portable Computing Language' at 0x5d4429f49320>:


(options: -I /home/leverlars/miniforge3/envs/pyopencl_env/lib/python3.12/site-packages/pyopencl/cl)

In [ ]:
mf = cl.mem_flags
vector_field = np.empty((N, M, 2), dtype=np.float32)
vector_field_buf = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=vector_field)
vector_field_derived_buf = cl.Buffer(context, mf.READ_WRITE, vector_field.nbytes)
angle_field = np.empty((N, M), dtype=np.float32)
angle_field_buf = cl.Buffer(context, mf.READ_WRITE, angle_field.nbytes)
scalar_field = np.empty((N, M), dtype=np.float32)
scalar_field_buf = cl.Buffer(context, mf.READ_WRITE, scalar_field.nbytes)

In [ ]:
seed = np.int32(69420)
create_vector_field_kernel = program.create_vector_field
create_vector_field_kernel.set_args(vector_field_buf, np.int32(N), np.int32(M), np.float32(s), seed)
cl.enqueue_nd_range_kernel(queue, create_vector_field_kernel, (N, M), None)

In [ ]:
# Simulation
num_steps = int(T / dt)
for step in range(num_steps):
    vector_field_derivative_kernel = program.vector_field_derivative
    vector_field_derivative_kernel.set_args(vector_field_buf, vector_field_derived_buf, np.int32(N), np.int32(M), np.float32(V_0), np.float32(Gamma))
    cl.enqueue_nd_range_kernel(queue, vector_field_derivative_kernel, (N, M), None)

    update_vector_field_kernel = program.update_vector_field
    update_vector_field_kernel.set_args(vector_field_buf, vector_field_derived_buf, np.float32(dt), np.int32(N), np.int32(M))
    cl.enqueue_nd_range_kernel(queue, update_vector_field_kernel, (N, M), None)

    cl.enqueue_copy(queue, vector_field, vector_field_buf)
    angle_from_field_kernel = program.angle_from_field
    angle_from_field_kernel.set_args(vector_field_buf, angle_field_buf, np.int32(N), np.int32(M), np.float32(s))
    cl.enqueue_nd_range_kernel(queue, angle_from_field_kernel, (N, M), None)
    scalar_from_field_kernel = program.scalar_from_field
    scalar_from_field_kernel.set_args(vector_field_buf, scalar_field_buf, np.int32(N), np.int32(M))
    cl.enqueue_nd_range_kernel(queue, scalar_from_field_kernel, (N, M), None)
    cl.enqueue_copy(queue, angle_field, angle_field_buf)
    cl.enqueue_copy(queue, scalar_field, scalar_field_buf)

    plt.figure(figsize=(5, 5))
    plt.imshow(np.linalg.norm(vector_field, axis=2), cmap='viridis')
    plt.colorbar()
    plt.title(f'Time Step {step}')
    plt.savefig(f'vector_field_images/vector_field_{step:03d}.png')
    plt.close()

    plt.figure(figsize=(5, 5))
    plt.imshow(angle_field, cmap='hsv')
    plt.colorbar()
    plt.title(f'Angle Field Time Step {step}')
    plt.savefig(f'angle_field_images/angle_field_{step:03d}.png')
    plt.close()

    plt.figure(figsize=(5, 5))
    plt.imshow(scalar_field, cmap='inferno')
    plt.colorbar()
    plt.title(f'Scalar Field Time Step {step}')
    plt.savefig(f'scalar_field_images/scalar_field_{step:03d}.png')
    plt.close()

In [ ]:
def create_gif(image_folder, gif_filename, duration=0.1):
    images = []
    for i in range(num_steps):
        filename = f'{image_folder}/{image_folder[:-7]}_{i:03d}.png'
        images.append(imageio.imread(filename))
    imageio.mimsave(gif_filename, images, duration=duration)

In [ ]:
create_gif('vector_field_images', 'vector_field_animation.gif')
create_gif('angle_field_images', 'angle_field_animation.gif')
create_gif('scalar_field_images', 'scalar_field_animation.gif')

In [ ]:
display(HTML("""
<div style="display: flex; justify-content: space-around;">
    <img src="vector_field_animation.gif" alt="Vector Field Animation">
    <img src="angle_field_animation.gif" alt="Angle Field Animation">
    <img src="scalar_field_animation.gif" alt="Scalar Field Animation">
</div>
"""))